### 필수과제 2
- 추가로 1개 또는 2개 이하로 다른 교차검증 패키지 사용해 보기
- 개념설명과 실제 적용해서 나온 결과값을 비교하기

- e.g) 타이타닉 데이터로 한다면 기존의 교차검증과 새롭게 진행할 교차검증의 차이를 비교해주세요!

In [3]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score #Confusion matrix
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

In [4]:
salary_df = pd.read_csv("salary_class.csv")

In [5]:
salary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      48842 non-null  int64 
 1   age             48842 non-null  int64 
 2   workclass       46043 non-null  object
 3   education       48842 non-null  object
 4   education-num   48842 non-null  int64 
 5   marital-status  48842 non-null  object
 6   occupation      46033 non-null  object
 7   relationship    48842 non-null  object
 8   race            48842 non-null  object
 9   sex             48842 non-null  object
 10  capital-gain    48842 non-null  int64 
 11  capital-loss    48842 non-null  int64 
 12  hours-per-week  48842 non-null  int64 
 13  native-country  47985 non-null  object
 14  class           48842 non-null  int64 
dtypes: int64(7), object(8)
memory usage: 5.6+ MB


In [6]:
#범주형데이터
categorical_col = ['workclass','education','marital-status','occupation','relationship','race','sex','native-country']
#연속형 데이터
num_col = ['age','education-num','capital-gain','capital-loss','hours-per-week','class']

In [7]:
from sklearn.preprocessing import LabelEncoder
enc_classes = {} 
def encoding_label(x):   # x: 범주형 타입의 컬럼(Series)
    le = LabelEncoder()
    le.fit(x)
    label = le.transform(x)
    enc_classes[x.name] = le.classes_   # x.name: 컬럼명
    return label

In [8]:
salary_encoding = salary_df[categorical_col].apply(encoding_label)
salary_not_encoding = salary_df[num_col]

In [9]:
salary_encoding_df=pd.concat([salary_encoding,salary_not_encoding],axis=1)

In [10]:
features = salary_encoding_df[['workclass','education','marital-status', 'occupation', \
                    'relationship','race','sex','native-country','age', 'education-num','hours-per-week']]

In [11]:
label=salary_encoding_df[['capital-gain']]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.2, random_state=111)

In [13]:
model = DecisionTreeClassifier()
model_sp1 = DecisionTreeClassifier(random_state=111, max_depth=3)

model.fit(X_train, y_train) #모델 학습

model_sp1.fit(X_train, y_train) #모델 학습

pred_train = model.predict(X_train) #train 예측
pred_test = model.predict(X_test) # test예측

pred_train_sp = model_sp1.predict(X_train) #train 예측
pred_test_sp = model_sp1.predict(X_test) # test예측

In [14]:
print(accuracy_score(y_train, pred_train_sp))
print(accuracy_score(y_test, pred_test_sp))

0.916387275100453
0.9213839697000716


In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.datasets import load_iris
import numpy as np

In [16]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=5) #데이터셋 분리

cv_accuracy_train = []
cv_accuracy_test = []
kf_model = DecisionTreeClassifier(random_state=111,max_depth=3)

In [17]:
n_iter = 0 #5번 진행하니 초깃값 설정

for train_idx, test_idx in kfold.split(features):
    X_train, X_test = features.iloc[train_idx], features.iloc[test_idx]
    y_train, y_test = label.iloc[train_idx], label.iloc[test_idx]
    
    #학습을 위해서 dt배우기
    kf_model.fit(X_train, y_train)
    #예측
    kf_pred_train = kf_model.predict(X_train)
    df_pred_test = kf_model.predict(X_test)
    
    #정확도를 5번 측정하면서 값에 넣을 것이니
    n_iter +=1
    accuracy_train = np.round(accuracy_score(y_train, kf_pred_train),4)
    accuracy_test = np.round(accuracy_score(y_test, df_pred_test),4)
    
    print('\n{} 번 train 교차 검증 정확도 :{} ,test 교차 검증 정확도 :{} '.format(n_iter,accuracy_train, accuracy_test))
    
    cv_accuracy_train.append(accuracy_train)
    cv_accuracy_test.append(accuracy_test)
    
print('train 평균 정확도',np.mean(cv_accuracy_train))
print('test 평균 정확도',np.mean(cv_accuracy_test))


1 번 train 교차 검증 정확도 :0.9171 ,test 교차 검증 정확도 :0.9184 

2 번 train 교차 검증 정확도 :0.9172 ,test 교차 검증 정확도 :0.9181 

3 번 train 교차 검증 정확도 :0.9175 ,test 교차 검증 정확도 :0.9169 

4 번 train 교차 검증 정확도 :0.9168 ,test 교차 검증 정확도 :0.9197 

5 번 train 교차 검증 정확도 :0.9183 ,test 교차 검증 정확도 :0.9138 
train 평균 정확도 0.91738
test 평균 정확도 0.91738


## 셔플분할 ShuffleSplit cross validation
- 전체 데이터셋을 무작위 추출하여 훈련/데이터 셋으로 나눈다

In [18]:
from sklearn.model_selection import ShuffleSplit
ss = ShuffleSplit(n_splits=5, test_size=0.33, random_state=0)
print("train\t\ttest")
for train_idx, test_idx in ss.split(features):
    X_train, X_test = features.iloc[train_idx], features.iloc[test_idx]
    y_train, y_test = label.iloc[train_idx], label.iloc[test_idx]

    # 학습을 위해 dt 모델을 피팅
    kf_model.fit(X_train, y_train)    
    
    # 예측
    kf_pred_train = kf_model.predict(X_train)
    df_pred_test = kf_model.predict(X_test)
    
    accuracy_train = np.round(accuracy_score(y_train, kf_pred_train), 4)
    accuracy_test = np.round(accuracy_score(y_test, df_pred_test), 4)
    
    print('Sample {} - train ShuffleSplit accuracy: {}, test ShuffleSplit accuracy: {}'.format(test_idx[0], accuracy_train, accuracy_test))
    
    cv_accuracy_train.append(accuracy_train)
    cv_accuracy_test.append(accuracy_test)
    
    
print('train 평균 정확도',np.mean(cv_accuracy_train))
print('test 평균 정확도',np.mean(cv_accuracy_test))

train		test
Sample 38113 - train ShuffleSplit accuracy: 0.9176, test ShuffleSplit accuracy: 0.9169
Sample 30326 - train ShuffleSplit accuracy: 0.9177, test ShuffleSplit accuracy: 0.9168
Sample 5635 - train ShuffleSplit accuracy: 0.918, test ShuffleSplit accuracy: 0.9161
Sample 45532 - train ShuffleSplit accuracy: 0.916, test ShuffleSplit accuracy: 0.9202
Sample 14769 - train ShuffleSplit accuracy: 0.9169, test ShuffleSplit accuracy: 0.9184
train 평균 정확도 0.91731
test 평균 정확도 0.91753


## 계층적 셔플분할
- 전체 데이터셋을 무작위 추출. 클래스 각각에 대해 비율이 유지되도록 함

## Leave-p-out 교차검증
- 전체 N개 샘플 데이터셋을 N-p개의 학습 데이터셋과 p개의 테스트 데이터셋으로 나눈다

In [ ]:
from sklearn.model_selection import LeavePOut
from sklearn.metrics import accuracy_score

p = 200

lpo = LeavePOut(p)

print("train\t\ttest")

for train_idx, test_idx in lpo.split(features):
    X_train, X_test = features.iloc[train_idx], features.iloc[test_idx]
    y_train, y_test = label.iloc[train_idx], label.iloc[test_idx]

    kf_model.fit(X_train, y_train)
    kf_pred_train = kf_model.predict(X_train)
    df_pred_test = kf_model.predict(X_test)
    
    accuracy_train = np.round(accuracy_score(y_train, kf_pred_train), 4)
    accuracy_test = np.round(accuracy_score(y_test, df_pred_test), 4)
    
    print('Leave-{}-Out - train accuracy: {}, test accuracy: {}'.format(p, accuracy_train, accuracy_test))


train		test
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.89
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out

Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.89
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.89
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.89
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.89
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accura

Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.89
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.89
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.89
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accur

Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.89
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.89
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accu

Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.89
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train acc

...

Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.89
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.89
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accu

Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.89
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.89
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accu

Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train ac

Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.89
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.89
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accu

Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train accuracy: 0.9175, test accuracy: 0.895
Leave-200-Out - train ac